In [ ]:
import os, getpass
from typing_extensions import Annotated, List
import operator
from pydantic import BaseModel, Field
from typing_extensions import TypedDict

# LLM og verktøy
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import FunctionTool
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate
from llama_index.core.llms import ChatMessage
from llama_index.core import (get_response_synthesizer)
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import (VectorStoreIndex, StorageContext,  load_index_from_storage)

# Import av langchain og langgraph
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.constants import Send
from langgraph.graph import StateGraph, START, END

# Indeksverktøy
LLMGPT4omini = AzureChatOpenAI(
    model=os.getenv('AZURE_OPENAI_MODEL_GPT4omini'),
    deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME_GPT4omini'),
    azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME_GPT4omini'),
    api_key=os.getenv('AZURE_OPENAI_API_KEY_GPT4omini'),
    azure_endpoint=os.getenv('AZURE_OPENAI_AZURE_ENDPOINT_GPT4omini'),
    api_version=os.getenv('AZURE_OPENAI_API_VERSJON_GPT4omini'),
    temperature=0.0,
    timeout= 120,
)

def read_index_from_storage(storage):
    storage_context = StorageContext.from_defaults(persist_dir=storage)
    return load_index_from_storage(storage_context)

# Sett Azure OpenAI-legitimasjon

llm = LLMGPT4omini


# Schema for structured output to use in planning
class SubQuery(BaseModel):
    subquery: str = Field(
        description="The subquery",
    )
    answer: str = Field(
        description="Answer to the subquery",
    )


class SubQueries(BaseModel):
    subqueries: List[SubQuery] = Field(
        description="Sections of the structured answer.",
    )


# Augment the LLM with schema for structured output
planner = llm.with_structured_output(SubQueries)



# Graph state
class State(TypedDict):
    query: str  # Report topic
    subqueries: list[SubQuery]  # List of subqueries
    completed_answers: Annotated[
        list, operator.add
    ]  # All workers write to this key in parallel
    final_answer: str  # Final report
    
# Worker state
class WorkerState(TypedDict):
    subquery: SubQuery
    completed_answers: Annotated[list, operator.add]
    
# some global variables
chat_text_qa_msgs = [
        ChatMessage(
            role=MessageRole.SYSTEM,
            content=( 
                "You are a helpful assistant, and you will be given a user request."
                "Some rules to follow:"
                "- Always answer the request using the given context information and not prior knowledge"
                "- Always answer in norwegian"
                "- Always answer using the format: '\033[33mAnswer: \033[34m<answer>\033[0m'"

            ),
        )
        ,
        ChatMessage(
            role=MessageRole.USER,
            content=(
                "Context information is below.\n"
                "---------------------\n"
                "{context_str}\n"
                "---------------------\n"
                "Query: {query_str}\n"
                "Answer: "
            ),
        ),
    ]
text_qa_template =  ChatPromptTemplate(chat_text_qa_msgs)

response_synthesizer = get_response_synthesizer(
    response_mode= "tree_summarize",
    text_qa_template = text_qa_template,
    summary_template= text_qa_template, #definitly in use for response_mode = tree_summarize
    structured_answer_filtering=True, 
    verbose=True,
)
text_splitter = SentenceSplitter.from_defaults(chunk_size=1024, chunk_overlap=75)



# Nodes
def orchestrator(state: State):
    """Orchestrator that generates a plan for solving the question"""
    # Generate queries
    report_queries = planner.invoke(
        [
            SystemMessage(content="Refrase the user query generating a subquery in norwegian. If the user query har several queries, generate several subqueries. Do not answer the subqueries"),
            HumanMessage(content=f"Here is query from a user: {state['query']}"),
        ]
    )
    return {"subqueries": report_queries.subqueries}

def llm_call(state: State):
    """Worker answer a subquery"""
    storage = './blobstorage/chatbot/ungnotobakk'
    if not state.get("index_tobakk"):
        state["index_tobakk"] = read_index_from_storage(storage)
    else:
        print(f'Index for tobakk already loaded from {storage}')

    query_engine = state["index_tobakk"].as_query_engine(
        similarity_cutoff=0.7, 
        similarity_top_k=10,
        response_synthesizer=response_synthesizer
    )

    response_obj = query_engine.query(state['subquery'].subquery)
    state['subquery'].answer = response_obj.response
    
    s = "\033[33mQuery: \033[34m" + state['subquery'].subquery + "\033[0m" + response_obj.response
    print(s)
    
    # Write the updated answer to completed sections
    return {"completed_answers": [state['subquery'].subquery + "\n" + response_obj.response]}
    
def synthesizer(state: State):
    """Synthesize full answer from answers from the subqueries"""

    # List of completed sections
    completed_answers = state["completed_answers"]

    # Format completed section to str to use as context for final sections
    completed_report_answers = "\n\n---\n\n".join(completed_answers)
    
    aggregated_answer = llm.invoke(
        [
            SystemMessage(
                content="from this list of answers, reorganize a final answer. Use markdown formatting."
            ),
            HumanMessage(
                content=f"Here is the list of answers: {completed_report_answers}"
            ),
        ]
    )
    
    #print(aggregated_answer.content)

    return {"final_answer": aggregated_answer.content}



# Conditional edge function to create llm_call workers that each write a section of the report
def assign_workers(state: State):
    """Assign a worker to each section in the plan"""

    # Kick off section writing in parallel via Send() API
    return [Send("llm_call", {"subquery": s}) for s in state["subqueries"]]



# Build workflow
orchestrator_worker_builder = StateGraph(State)

# Add the nodes
orchestrator_worker_builder.add_node("orchestrator", orchestrator)
orchestrator_worker_builder.add_node("llm_call", llm_call)
orchestrator_worker_builder.add_node("synthesizer", synthesizer)

# Add edges to connect nodes
orchestrator_worker_builder.add_edge(START, "orchestrator")
orchestrator_worker_builder.add_conditional_edges(
    "orchestrator", assign_workers, ["llm_call"]
)
orchestrator_worker_builder.add_edge("llm_call", "synthesizer")
orchestrator_worker_builder.add_edge("synthesizer", END)

# Compile the workflow
orchestrator_worker = orchestrator_worker_builder.compile()



from graph_utils import save_mermaid_diagram
save_mermaid_diagram(orchestrator_worker.get_graph())
# Show the workflow
#display(Image(router_workflow.get_graph().draw_mermaid_png()))

# Invoke
state = orchestrator_worker.invoke({"query": "Kan du gi meg tips på hvordan slutteå røyke? Når sank titanic? Er snus skadelig for min helse?"})

from IPython.display import Markdown
Markdown(state["final_answer"])



llm_call invoked: Worker answer a subquery

llm_call invoked: Worker answer a subquery

llm_call invoked: Worker answer a subquery
1 text chunks after repacking
1 text chunks after repacking
Query: Når sank Titanic?Answer: Dette spørsmålet omhandler ikke informasjonen som er gitt i konteksten.
1 text chunks after repacking
Query: Er snus skadelig for min helse?Answer: Ja, snus er skadelig for helsen din. Snus inneholder helseskadelige stoffer og kan føre til ulike helseproblemer som kreft, hjerteproblemer, diabetes og påvirke utviklingen av hjernen. Det er spesielt farlig for gravide å bruke snus, da det kan påvirke fosteret negativt. Det anbefales å unngå snus for å opprettholde god helse.
Query: Kan du gi meg tips på hvordan slutte å røyke?Answer: For å slutte å røyke kan du følge disse tipsene:

1. Finn noe som motiverer deg til å slutte. Skriv ned alle gode grunner for å slutte og ha de synlige hver dag.
2. Sett en sluttdato og ta hensyn til abstinenser i tiden etter nikotinslutt.

## Tips for å slutte å røyke

For å slutte å røyke kan du følge disse tipsene:

1. **Motivasjon**: Finn noe som motiverer deg til å slutte. Skriv ned alle gode grunner for å slutte og ha dem synlige hver dag.
2. **Sluttdato**: Sett en sluttdato og vær forberedt på abstinenser i tiden etter nikotinslutt.
3. **Støtte**: Finn støtte i andre. Del med venner og familie at du har bestemt deg for å slutte.
4. **Verktøy**: Bruk Slutta-appen for daglige motivasjonsmeldinger og tips.
5. **Belønning**: Belønn deg selv når du når delmål, for eksempel med en hyggelig aktivitet eller en liten premie.

Lykke til med røykeslutt!

---

## Er snus skadelig for helsen?

Ja, snus er skadelig for helsen din. Snus inneholder helseskadelige stoffer og kan føre til ulike helseproblemer som kreft, hjerteproblemer, diabetes, og påvirke utviklingen av hjernen. Det er spesielt farlig for gravide å bruke snus, da det kan påvirke fosteret negativt. Det anbefales å unngå snus for å opprettholde god helse.